1. Check if Video is 360p, if not convert to 360p
2. Use psv files and json to only target sections of podcast where guest is speaking
    (do not want biased joe rogan data)
3. Extract 4 second clips at 10fps gray scale gifs
4. Filter out any clips that have a 'scene change' aka the camera pans to the listener

In [ ]:
#Global variables
import socket
import os
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data\\'
ENV_FOLDER_DATA_PROC = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data_Proc\\'
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

In [ ]:
#!pip install ffmpeg-python

import ffmpeg

input_file = ENV_FOLDER_DATA  + 'Joe Rogan Experience #568 - Dr. Rhonda Patrick.mp4'

def get_video_resolution(video_file):
    probe = ffmpeg.probe(video_file)
    video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
    if video_stream:
        width = int(video_stream['width'])
        height = int(video_stream['height'])
        resolution = (width, height)
        return resolution
    else:
        return None

def is_360p(video_file):
    resolution = get_video_resolution(video_file)
    if resolution:
        width, height = resolution
        if width <= 640 and height <= 360:
            return True
    return False

def convert_to_360p(video_file, output_file):
    ffmpeg.input(video_file).output(output_file, s='640x360').run()


In [ ]:
import cv2
import glob

def detect_scene_changes(gif_path, threshold=10):
    gif = cv2.VideoCapture(gif_path)
    frame_rate = gif.get(cv2.CAP_PROP_FPS) # Get frame rate
    prev_frame = None
    scene_changes = []

    frame_number = 0
    while True:
        ret, frame = gif.read()
        if not ret:
            break
        
        # Convert frame to grayscale for simplicity
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        if prev_frame is not None:
            # Calculate frame difference
            frame_diff = cv2.absdiff(gray_frame, prev_frame)
            diff_score = frame_diff.mean()
            
            # Check if scene change detected
            if diff_score > threshold:
                # Calculate time in seconds
                time_in_seconds = frame_number / frame_rate
                scene_changes.append((time_in_seconds, diff_score))
        
        prev_frame = gray_frame
        frame_number += 1

    return scene_changes

# Example usage

# for file in glob.glob(ENV_FOLDER_DATA_PROC + 'targetclips\\*.gif'):
#     scene_changes = detect_scene_changes(file)
#     print("Scene changes for", file)
#     for change in scene_changes:
#         print("Time:", change[0], "seconds, Score:", change[1])
# gif_path = ENV_FOLDER_DATA_PROC + "targetclips\\208_clip1479.3-1502.9.gif"
# scene_changes = detect_scene_changes(gif_path)
# print("Scene changes:")
# for change in scene_changes:
#     print("Time:", change[0], "seconds, Score:", change[1])



In [ ]:
# Delete clips where scene changes are detected unless the scene changes within the first 5 seconds or the last 5 seconds

def get_clip_length(filename):
    # Extract start and end timestamps from filename
    start, end = filename.split("_clip")[1].split(".gif")[0].split("-")
    
    # Convert timestamps to float
    start = float(start)
    end = float(end)
    
    # Calculate clip length
    clip_length = end - start
    
    return clip_length

def delete_clips_with_scene_changes(gif_path, threshold=10):

    for file in glob.glob(gif_path):
        scene_changes = detect_scene_changes(file)
        if len(scene_changes) == 0:
            continue

        clip_length = get_clip_length(file)
        
        first_change = scene_changes[0]
        last_change = scene_changes[-1]

        if len(scene_changes) < 2:
            if first_change[0] < 5 or last_change[0] > clip_length - 5:
                #print("Scene change detected within first or last 5 seconds")
                continue

        #print("Deleting", file)
        os.remove(file)

# Example usage
#delete_clips_with_scene_changes(ENV_FOLDER_DATA_PROC + "targetclips\\*.gif")